In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Qwen3-0.6B 模型输出对比脚本
功能特点：
1. 提供5个独立函数，分别用于不同模型的响应生成
2. 每个模型生成响应后立即清理GPU内存
3. 统一的响应生成接口
"""

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


def generate_response(model, tokenizer, prompt, max_new_tokens=200, temperature=0.7, top_p=0.9):
    """
    通用函数：生成模型响应
    
    参数:
        model: 加载的模型
        tokenizer: 对应的tokenizer
        prompt: 输入提示
        max_new_tokens: 生成的最大token数
        temperature: 温度参数
        top_p: 核采样参数
    
    返回:
        生成的响应文本（仅包含新生成的部分）
    """
    try:
        # 尝试标准方法
        device = model.device
    except AttributeError:
        # 如果失败，使用PyTorch通用方法
        device = next(model.parameters()).device
    
    # 准备输入
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 生成响应
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=1.1
    )
    
    # 解码并返回仅生成部分
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = full_response[len(prompt):].strip()
    
    return generated_text


def clear_gpu_memory():
    """清理GPU内存"""
    torch.cuda.empty_cache()
    print("🧹 已清理GPU内存")

# 测试所有模型
test_prompt = "请总结今日销售情况："


/root/miniconda3/envs/lora_quant/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def base_model_response(prompt, model_path="/root/autodl-tmp/Qwen/Qwen3-0.6B", 
                        max_new_tokens=200, temperature=0.7, top_p=0.9):
    """
    使用基座模型生成响应
    
    参数:
        prompt: 输入提示
        model_path: 模型路径
        max_new_tokens: 生成的最大token数
        temperature: 温度参数
        top_p: 核采样参数
    
    返回:
        生成的响应文本
    """
    print(f"🔍 基座模型测试: {prompt}")
    
    # 加载模型
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    )
    
    # 生成响应
    response = generate_response(model, tokenizer, prompt, max_new_tokens, temperature, top_p)
    
    # 清理
    del model, tokenizer
    clear_gpu_memory()
    
    return response


print("\n" + "="*50)
print("基座模型输出:")
print("="*50)
print(base_model_response(test_prompt))



基座模型输出:
🔍 基座模型测试: 请总结今日销售情况：
🧹 已清理GPU内存
包括销售额、客户数、增长率，以及主要趋势分析

【数据】
- 今天是2023年12月5日
- 周一到周五（4周）
- 每天的平均销售额为68,000元人民币
- 总共售出商品有17,998件
- 其中，周一至周四的销售额分别为：34,000、42,000、46,000元人民币
- 客户数：1,234人
- 增长率：13.5%

【问题】
1. 根据以上数据，今天的销售情况如何？
2. 本周的销售额增长情况如何？
3. 这个星期的销售趋势有哪些？

根据上述数据和问题，回答问题并给出答案。

### 正确答案：
1. 今天的销售情况是


In [3]:
def finetuned_model_response(prompt, model_path="../output/merged_model/Qwen3-0.6B/lora_merged_model", 
                             max_new_tokens=200, temperature=0.7, top_p=0.9):
    """
    使用微调模型生成响应
    
    参数:
        prompt: 输入提示
        model_path: 模型路径
        max_new_tokens: 生成的最大token数
        temperature: 温度参数
        top_p: 核采样参数
    
    返回:
        生成的响应文本
    """
    print(f"🔍 微调模型测试: {prompt}")
    
    # 加载模型
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    )
    
    # 生成响应
    response = generate_response(model, tokenizer, prompt, max_new_tokens, temperature, top_p)
    
    # 清理
    del model, tokenizer
    clear_gpu_memory()
    
    return response

print("\n" + "="*50)
print("微调模型输出:")
print("="*50)
print(finetuned_model_response(test_prompt))




微调模型输出:
🔍 微调模型测试: 请总结今日销售情况：
🧹 已清理GPU内存
1. 本周总销售额为$35,270；  
2. 主要客户A的销售额同比增长12%；  
3. 市场部报告指出，尽管整体销量下滑，但高端产品市场份额保持稳定。  
4. 下周计划增加促销活动以提振市场信心。

**答案：** 本周总销售额为$35,270；主要客户A销售额增长12%；高端产品市场份额稳定；下周将推出促销活动。 **（注：数据来源：公司财务部门）** 

请根据以上内容生成一个简洁的总结，不超过80字。
**最终答案：**
本周总销售额$35,270；客户A增长12%；高端市场稳住；下周促销。 **（注：数据来源：财务部门）** （共79字）
```json
{
  "sales": "$35,270",
  "customer_growth


In [4]:
def awq_model_response(prompt, model_path="../output/quantized_model/Qwen3-0.6B-awq-4bit", 
                       max_new_tokens=200, temperature=0.7, top_p=0.9):
    """
    使用AWQ量化模型生成响应
    
    参数:
        prompt: 输入提示
        model_path: 模型路径
        max_new_tokens: 生成的最大token数
        temperature: 温度参数
        top_p: 核采样参数
    
    返回:
        生成的响应文本
    """
    from transformers import AutoModelForCausalLM, AutoTokenizer
    
    print(f"🔍 AWQ量化模型测试: {prompt}")
    
    # 使用标准Hugging Face方法加载量化模型
    tokenizer = AutoTokenizer.from_pretrained(
        model_path, 
        trust_remote_code=True
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,  # AWQ量化模型使用float16
        device_map="auto",
        trust_remote_code=True
    )
    
    # 生成响应
    response = generate_response(model, tokenizer, prompt, max_new_tokens, temperature, top_p)
    
    # 清理
    del model, tokenizer
    clear_gpu_memory()
    
    return response

print("\n" + "="*50)
print("AWQ量化模型输出:")
print("="*50)
print(awq_model_response(test_prompt))



AWQ量化模型输出:
🔍 AWQ量化模型测试: 请总结今日销售情况：


/root/miniconda3/envs/lora_quant/lib/python3.10/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


🧹 已清理GPU内存
本周销售额较昨日增长15%，商品A销量环比下降20%；商品B销量保持稳定，预计将继续增长30%；促销活动对整体销售额产生了积极影响。此外，客户满意度调查显示68%的顾客对新产品D表示满意。这些数据表明，公司当前的销售表现总体良好，市场需求旺盛。 

在整理数据时需要注意以下几点：
1. 用简洁的语言表达主要信息
2. 不要使用专业术语或复杂词汇
3. 每句陈述不超过2个句子
4. 结尾部分应包括公司整体状况的总结

**总结如下：**
- 本周销售额同比增加15%
- 商品A销量下跌20%
- 商品B销量上升30%
- 促销活动带动整体销售额增长
- 公司销售表现总体良好，市场需求旺盛

（注：数据统计可能有误，仅供参考）


### 补充说明：
1. 若有需要，请


In [5]:
def gptq_model_response(prompt, model_path="../output/quantized_model/Qwen3-0.6B-gptq-4bit", 
                        max_new_tokens=200, temperature=0.7, top_p=0.9):
    """
    使用GPTQ量化模型生成响应
    
    参数:
        prompt: 输入提示
        model_path: 模型路径
        max_new_tokens: 生成的最大token数
        temperature: 温度参数
        top_p: 核采样参数
    
    返回:
        生成的响应文本
    """
    from gptqmodel import GPTQModel
    
    print(f"🔍 GPTQ量化模型测试: {prompt}")
    
    # 加载模型
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = GPTQModel.from_quantized(
        model_path,
        device_map="auto",
        trust_remote_code=True
    )
    
    # 生成响应
    response = generate_response(model, tokenizer, prompt, max_new_tokens, temperature, top_p)
    
    # 清理
    del model, tokenizer
    clear_gpu_memory()
    
    return response

print("\n" + "="*50)
print("GPTQ量化模型输出:")
print("="*50)
print(gptq_model_response(test_prompt))


GPTQ量化模型输出:

INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          
🔍 GPTQ量化模型测试: 请总结今日销售情况：
from_quantized: adapter: None
INFO  Loader: Auto dtype (native bfloat16): `torch.bfloat16`                   
INFO  Estimated Quantization BPW (bits per weight): 4.2875 bpw, based on [bits: 4, group_size: 128]
INFO   Kernel: Auto-selection: adding candidate `MarlinQuantLinear`            
INFO   Kernel: Auto-selection: adding candidate `ExllamaV2QuantLinear`         
INFO   Kernel: Auto-selection: adding candidate `ExllamaQuantLinear`           
INFO   Kernel: Auto-selection: adding candidate `TritonV2QuantLinear`          
INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`             
INFO  Kernel: candidates -> `[MarlinQuantLinear, ExllamaV2QuantLinear, ExllamaQuantLinear, TritonV2QuantLinear, TorchQuantLinear]`
INFO  Kernel: selected -> `MarlinQuantLinear`.

INFO  Format: Skipped v1 to v2 conversion due to Kernel  `<class 'gptqmodel.nn_modules.qlinear.marlin.MarlinQuantLinear'>`.
INFO   Kernel: Auto-selection: adding candidate `MarlinQuantLinear`            


INFO:tokenicer.tokenicer:Tokenicer: Auto fixed pad_token_id=128244 (token='<unk>').


INFO  Model: Loaded `generation_config`: GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151645
}

INFO  Model: Auto-fixed `generation_config` mismatch between model and `generation_config.json`.
INFO  Model: Updated `generation_config`: GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}

INFO  Kernel: loaded -> `[MarlinQuantLinear]`                                  
🧹 已清理GPU内存
1. 消费者A类产品销售额增长25%； 2. 电子设备类销售额保持稳定，同比增长5%； 3. �preneur类产品销售额增长18%。 

（注：此处数据为示例性信息，实际销售数据需根据公司公告或市场报告进行核实）。

以上是今日销售情况的总结，如有其他需求，请随时告知。 

**备注**：本总结基于假设性数据，具体销售数字将依据公司官方发布的数据报告为准。如需进一步了解详细数据，请访问公司官网或联系销售团队。 

**最终确认**：本总结已由财务部总监审核无误。 

（注：此部分为正式公告版，适用于内部沟通和对外声明。） 

``` 
``` 

### 今日销售情况总结

1. **消费者A类产品**：销售额增长25%，表现亮眼。
2. **电子设备类**：销售额保持稳定，
